# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [ ]:
import pandas as pd

# URLs for the CSV files
url1 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv'
url2 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv'
url3 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv'

# Load the data from URLs
df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)
df3 = pd.read_csv(url3)

# Print column names for verification
print("Columns in df1:")
print(df1.columns.tolist())

print("\nColumns in df2:")
print(df2.columns.tolist())

print("\nColumns in df3:")
print(df3.columns.tolist())

# Standardize column names
df3.rename(columns={'State': 'ST', 'Gender': 'GENDER'}, inplace=True)

# Combine df2 and df3 into df1
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Create a new column with sequential ID
combined_df['ID'] = range(1, len(combined_df) + 1)

# Remove duplicate rows
combined_df = combined_df.drop_duplicates()

# Remove leading and trailing spaces from column names
combined_df.columns = combined_df.columns.str.strip()

# Remove leading and trailing spaces from text values in columns
for col in ['Customer', 'Education', 'Policy Type', 'Vehicle Class', 'GENDER']:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].str.strip()

# Clean 'Customer Lifetime Value' by removing percentage symbols and converting to float
if 'Customer Lifetime Value' in combined_df.columns:
    combined_df['Customer Lifetime Value'] = combined_df['Customer Lifetime Value'].str.replace('%', '').astype(float)

# Clean 'Number of Open Complaints' by removing non-numeric characters and converting to integers
if 'Number of Open Complaints' in combined_df.columns:
    combined_df['Number of Open Complaints'] = (
        combined_df['Number of Open Complaints']
        .str.replace(r'[^0-9]', '', regex=True)  # Remove non-numeric characters
        .replace('', '0')  # Replace empty strings with '0'
        .fillna('0')  # Replace NaN with '0'
        .astype(int)
    )

# Fill missing values
combined_df = combined_df.fillna({
    'Customer Lifetime Value': 0,
    'Income': 0,
    'Monthly Premium Auto': 0,
    'Number of Open Complaints': 0,
    'Total Claim Amount': 0
})

# Convert numerical columns to appropriate types
combined_df['Income'] = combined_df['Income'].astype(float)
combined_df['Monthly Premium Auto'] = combined_df['Monthly Premium Auto'].astype(float)
combined_df['Total Claim Amount'] = combined_df['Total Claim Amount'].astype(float)

# Print the cleaned DataFrame with ID
print("\nCleaned DataFrame with ID:")
print(combined_df.head())




# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [1]:
# Your code goes here

import pandas as pd

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'

db = pd.read_csv(url)

print(db.info())
print(db.columns.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unnamed:_0                     10910 non-null  int64  
 1   customer                       10910 non-null  object 
 2   state                          10910 non-null  object 
 3   customer_lifetime_value        10910 non-null  float64
 4   response                       10910 non-null  object 
 5   coverage                       10910 non-null  object 
 6   education                      10910 non-null  object 
 7   effective_to_date              10910 non-null  object 
 8   employmentstatus               10910 non-null  object 
 9   gender                         10910 non-null  object 
 10  income                         10910 non-null  int64  
 11  location_code                  10910 non-null  object 
 12  marital_status                 10910 non-null 

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [ ]:
pivot_revenue = db.pivot_table( values='total_claim_amount', index='sales_channel', aggfunc='sum').round(2)

print('Table:', pivot_revenue)

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [ ]:
pivot_average_lifetime = db.pivot_table( values='customer_lifetime_value', index='sales_channel', aggfunc='mean').round(2)

print('Table:', pivot_average_lifetime)

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [5]:
# Your code goes here

summary_table = db.pivot_table(values='number_of_open_complaints', index=['policy_type', 'month'], aggfunc='count').reset_index()

# Rename columns for clarity
summary_table.columns = ['Policy Type', 'Month', 'Number of complaints']

print(summary_table)

      Policy Type  Month  Number of Complaints
0  Corporate Auto      1                  1252
1  Corporate Auto      2                  1089
2   Personal Auto      1                  4329
3   Personal Auto      2                  3799
4    Special Auto      1                   237
5    Special Auto      2                   204
